# Creating a Operator

In [5]:
import sys
sys.path.insert(0, '../') 
from dotenv import load_dotenv
load_dotenv()
%load_ext autoreload
%autoreload 2

In [6]:
# Use the different modules to create an operator instance
from openoperator import OpenOperator
from openoperator import BlobStore
from openoperator import DocumentLoader
from openoperator import VectorStore
from openoperator import Embeddings
from openoperator import KnowledgeGraph
from openoperator import LLM

# Create the different modules that are needed for the operator
blob_store = BlobStore()
document_loader = DocumentLoader() # used to extract text from documents
embeddings = Embeddings()
vector_store = VectorStore(embeddings=embeddings)
knowledge_graph = KnowledgeGraph()
llm = LLM(model_name="gpt-4-0125-preview")

operator = OpenOperator(
    blob_store=blob_store,
    document_loader=document_loader,
    vector_store=vector_store,
    embeddings=embeddings,
    knowledge_graph=knowledge_graph,
    llm=llm
)

## Create a Portfolio and a Facility

In [7]:
portfolio = operator.create_portfolio('Setty & Associates test 4')
facility = portfolio.create_facility('DC Office test 5')

## Lets load all the files for the facility

In [8]:
import mimetypes
import os
directory_path = "setty_office_data/documents/" 

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        file_type = mimetypes.guess_type(file_path)[0]
        with open(file_path, 'rb') as file:
            file_content = file.read()
            facility.upload_document(file_content, filename, file_type=file_type)

## Now we can search through all the data within those documents

In [9]:
# Search for metadata in documents
docs = facility.search_documents("AHU", limit=2)
docs

[{'content': 'AHU SCHEDULE\n\nOmniClass Number\n\n23.33.39.17.13.11 23.33.39.17.13.11\n\nOmniClass Title\n\nAir Cooled Single Pack Packaged Air Conditioners Air Cooled Single Pack Packaged Air Conditioners\n\nMark\n\nAHU-1 AHU-2\n\nFuel Type\n\nNATURAL GAS NATURAL GAS\n\nInternal Efficiency (%) 82 82\n\nMax External Static Pressure 1" W.C 1" W.C\n\nCFM (nominal) Heating Input Max.\n\n0 CFM 0 CFM\n\n224000 BTU/Hr 224000 BTU/Hr\n\ncooling\n\nHeating Output Capacity\n\n140000 BTU/Hr 184000 BTU/Hr 140000 BTU/Hr 184000 BTU/Hr\n\nEER\n\n10.8 10.8\n\nCOMPRESSOR A LRA 123 123',
  'metadata': {'file_url': 'https://syystorage.blob.core.windows.net/open-operator-2/2023-12-30_DC%20Office_Mechanical.pdf',
   'filename': '2023-12-30_DC Office_Mechanical.pdf',
   'filetype': 'application/pdf',
   'languages': ['eng'],
   'facility_id': 'd40df347-fa19-4736-a00e-6fc12dfda66c',
   'page_number': 7,
   'portfolio_id': '202f7072-5687-4a1c-b631-748d5dfe5262'}},
 {'content': 'AC AF AFF AHU AI AO APD ARCH AS

## Upload COBie spreadsheets

In [10]:
import os
directory_path = "setty_office_data/cobie_sheets/"

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'rb') as file:
            file_content = file.read()
            errors_found, errors = facility.upload_cobie_spreadsheet(file_content)
            if errors_found:
                print(f"Errors found in {filename}: {errors}")

Errors found in CObie_Mechanical.xlsx: {'More than one record found in Facility sheet.': [{'sheet': 'Facility', 'row': 1, 'column': 1}], 'Not every Type record has a category.': [{'sheet': 'Type', 'row': 14, 'column': 4}, {'sheet': 'Type', 'row': 17, 'column': 4}, {'sheet': 'Type', 'row': 24, 'column': 4}, {'sheet': 'Type', 'row': 30, 'column': 4}, {'sheet': 'Type', 'row': 31, 'column': 4}, {'sheet': 'Type', 'row': 32, 'column': 4}, {'sheet': 'Type', 'row': 33, 'column': 4}, {'sheet': 'Type', 'row': 45, 'column': 4}, {'sheet': 'Type', 'row': 46, 'column': 4}, {'sheet': 'Type', 'row': 47, 'column': 4}, {'sheet': 'Type', 'row': 52, 'column': 4}], 'Component is not linked to an existing Space.': [{'sheet': 'Component', 'row': 28, 'column': 5}, {'sheet': 'Component', 'row': 29, 'column': 5}, {'sheet': 'Component', 'row': 34, 'column': 5}, {'sheet': 'Component', 'row': 56, 'column': 5}, {'sheet': 'Component', 'row': 63, 'column': 5}, {'sheet': 'Component', 'row': 64, 'column': 5}, {'sheet':

## Chat with the AI Assistant

In [6]:
messages = [
    {
        "role": "user",
        "content": "create a table of my air handling units schedule"
    }
]
for response in operator.chat(messages, portfolio, facility, verbose=True):
    print(response, flush=True, end="")

Tool Selected: search_building_documents
Tool args: {'query': 'air handling units schedule'}
Tool response:
[
  {
    "content": "AIR DEVICE SCHEDULE\n\nOmniClass Number\n\n23.33.43.15\n\n23.33.43.15\n\n23.33.43.15\n\nOmniClass Title Refrigeration Condenser Units Refrigeration Condenser Units Refrigeration Condenser Units\n\nMark\n\nACCU-1\n\nACCU-2\n\nACCU-3\n\nManufacturer\n\nMitsubishi Electric\n\nMitsubishi Electric\n\nMitsubishi Electric\n\nOmniClass Number\n\n23.33.39.21\n\n23.33.39.21\n\n23.33.39.21\n\nOmniClass Title Split System Air Conditioning Units Split System Air Conditioning Units Split System Air Conditioning Units\n\nMark\n\nManufacturer\n\nAC-1 Mitsubishi Electric",
    "metadata": {
      "file_url": "https://syystorage.blob.core.windows.net/open-operator-2/2023-12-30_DC%20Office_Mechanical.pdf",
      "filename": "2023-12-30_DC Office_Mechanical.pdf",
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "facility_id": "e8abbc6b-2979